In [12]:
import requests
import json
import pandas as pd
import math
from IPython.display import Image, display, clear_output


In [2]:
def readJson(file):
    with open(file,"r") as f:
        rf=json.load(f)
    return rf

def readTsv(file):
    with open(file, "r") as f:
        rf=csv.DictReader(f, dialect='excel-tab')
    return rf

In [3]:
# Initial requests from API url (23.08.19)
config = readJson("config.json")
client_id=config["client_id"]
elev_key = config["elevation_key"]
test_area = config["test_area"]

def requestImagesBySequence(sequence_key):
    request = requests.get("https://a.mapillary.com/v3/images/?sequence_keys="+sequence_key+"&per_page=10000&client_id="+client_id)
    return request.json()

In [4]:
# data = readJson("mapillary_data.json")
feature_names = ["avg_speed","max_speed","distance", "duration", "id"]
data = pd.read_csv("training_data/features.csv", names=feature_names)
data

,avg_speed,max_speed,distance,duration,id
0,16.092218,25.961550,7917.451715,492.005,6lmxhlsdu2820pp9ewk41z
1,6.322339,12.872192,1132.324604,179.099,g234mtreigui4a64i4xea9
2,1.275309,2.196088,366.273853,287.204,hwt93drmgmyk3pnt7hib08
3,1.210824,2.395365,562.095759,464.226,9gmnn1cdfv7vbptl9p83u2
4,1.313297,2.017232,126.595226,96.395,18ejk5cjxixj051qiupxyb
...,...,...,...,...,...
2693,1.262569,4.445656,508.815282,403.000,685p3jdyxdzwqswrdiznr3
2694,1.237501,4.137055,2234.277163,1805.475,ZJHnYlGbTkqUfi4DAxWBKw
2695,1.291060,12.306978,679.097605,526.000,Jrj8SnPk8c08pvkAZ_gEyw
2696,1.691342,7.648178,717.128975,424.000,vUubFKacybDCobluHgcKUw


In [5]:
data.iloc[0]["avg_speed"]

16.092217995524038

In [ ]:
index_to_start = pd.read_csv("sequence_labels.csv", header=None).shape[0]
f = open("sequence_labels.csv", "a+")
a = ""
num = data.shape[0]
for i, seq in enumerate(data["id"][index_to_start:], index_to_start):
    print(f"[INFO]\nprogress:{i}/{num}\n")
    images = requestImagesBySequence(seq)["features"]
    if(a=="q"):
        break
    interval = math.ceil(len(images)/5)
    for j in range(0,len(images),interval):
        clear_output()
        print(i)
        key = images[j]["properties"]["key"]
        display(Image(url= f"https://images.mapillary.com/{key}/thumb-320.jpg"))
        a = input(f"0 = ped, 1 = bic, 2 = car, q=quit, anything_else = next image\n{data.iloc[i]}")
        if(a=="q"):
            break
        elif(len(a)>0 and (int(a)==0 or int(a)==1 or int(a)==2)):
            f.write(f"{a},{seq}\n")
            break
        else:
            if(j+interval>len(images)):
                print("No type chosen!!!\n")
                while(True):
                    a = input(f"0 = ped, 1 = bic, 2 = car, q=quit")
                    if(len(a)>0 and (int(a)==0 or int(a)==1 or int(a)==2)):
                        f.write(f"{a},{seq}\n")
                        break
            else:
                continue
f.close()